In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os, re
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence, clean_text_to_sequence, clean_text

from scipy.io.wavfile import write

In [2]:
def get_text(text, hps, cleaned=False):
    text_norm = text_to_sequence(text, hps.symbols, [])
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

In [3]:
print(torch.__version__)           # 1.12.1
print(torch.version.cuda)           # 11.3
print(torch.cuda.is_available())     #True

1.12.1
11.3
True


In [4]:
configDir = "./configs/config_CNJP_cleaners_4p.json"
modelDir = "./models/config_CNJP_cleaners_4p.pth"

In [5]:
hps = utils.get_hparams_from_file(configDir)

In [6]:
net_g = SynthesizerTrn(
    len(hps.symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model).cuda()
_ = net_g.eval()

_ = utils.load_checkpoint(modelDir, net_g, None)

In [16]:
text = "[ZH]臭羊，你这个[ZH][JA]へんたアアい！！！[JA]"
speakerID = 0
length_scale = 1

text_cleaned = clean_text(text, hps.data.text_cleaners)
# text_cleaned = _clean_text(text, hps.data.text_cleaners)
print(hps.data.text_cleaners)
print(text_cleaned)


['zh_ja_mixture_cleaners']
ʦ`ʰou↓yaNg↑, ni↓↑ ʦ`⁼ə↓k⁼ə↓ he↓Nta a↓ai!!!


In [17]:
stn_tst = get_text(text_cleaned, hps)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    sid = torch.LongTensor([0]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667, noise_scale_w=0.8, length_scale=length_scale)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

In [9]:
dataset = TextAudioSpeakerLoader(hps.data.validation_files, hps.data)
collate_fn = TextAudioSpeakerCollate()
loader = DataLoader(dataset, num_workers=8, shuffle=False,
    batch_size=1, pin_memory=True,
    drop_last=True, collate_fn=collate_fn)
data_list = list(loader)

AttributeError: 'HParams' object has no attribute 'validation_files'